In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Configuring the device to use GPU if available, else CPU:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setting hyperparameters: number of training epochs, batch size, and learning rate:
num_epochs = 25
batch_size = 100
learning_rate = 0.001

# Defining image preprocessing steps for augmentation and normalization:
transform = transforms.Compose([
    transforms.Pad(4),                  # Padding images
    transforms.RandomHorizontalFlip(),  # Random horizontal flipping
    transforms.RandomCrop(32),          # Random cropping to 32x32
    transforms.ToTensor()])             # Converting to tensor

# Loading and transforming the CIFAR-10 training dataset:
train_dataset = torchvision.datasets.CIFAR10(root='/data',
                                             train=True, 
                                             transform=transform,
                                             download=True)

# Loading and transforming the CIFAR-10 test dataset:
test_dataset = torchvision.datasets.CIFAR10(root='/data',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Creating data loaders for batching, shuffling training data:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

# Creating data loaders for test data (no shuffling needed):
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

100%|██████████████████████████████████████████████████████████████| 170498071/170498071 [00:05<00:00, 30477653.41it/s]


Extracting /data\cifar-10-python.tar.gz to /data


![resnetimage](https://user-images.githubusercontent.com/30661597/78585170-f4ac7c80-786b-11ea-8b00-8b751c65f5ca.PNG)

In [3]:
# Function to create a 3x3 convolutional layer with specified in/out channels and stride:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

# Class defining a Residual Block:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        
        # First convolutional layer in the residual block:
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        
        # Batch normalization following the first convolution:
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        # ReLU activation function:
        self.relu = nn.ReLU(inplace=True)
        
        # Second convolutional layer in the residual block:
        self.conv2 = conv3x3(out_channels, out_channels)
        
        # Batch normalization following the second convolution:
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Downsampling function if provided (used in case of increasing dimensions):
        self.downsample = downsample
        
        
    def forward(self, x):
        # Storing the original input for the skip connection:
        residual = x
        
        # Passing input through the first conv layer, batch norm, and ReLU:
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        # Passing through the second conv layer and batch norm:
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Applying downsampling to the residual if needed:
        if self.downsample:
            residual = self.downsample(x)
            
        # Adding the residual (skip connection):
        out += residual
        
        # Final ReLU activation:
        out = self.relu(out)
        
        return out

In [4]:
# Defining the ResNet class:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        
        # Initial convolutional layer parameters:
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        
        # Creating layers of residual blocks:
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        
        # Average pooling and fully connected layer:
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
     
    # Function to create layers of residual blocks:
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        
        # Downsampling for dimensionality alignment:
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(conv3x3(self.in_channels, out_channels, stride=stride),
                                       nn.BatchNorm2d(out_channels))
        layers = []
        
        # Adding the first block (with potential downsampling):
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        
        # Adding remaining blocks:
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    # Defining the forward pass:
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        
        # Passing input through residual layers:
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        
        # Applying average pooling and flattening:
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [5]:
# Initializing the ResNet model with the ResidualBlock and 2 blocks in each layer:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

# Setting up the Cross Entropy Loss function for classification:
criterion = nn.CrossEntropyLoss()

# Choosing the Adam optimizer with the defined learning rate:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Learning rate decay initialization:
decay = 0

# Setting the model to training mode:
model.train()

# Training loop for the specified number of epochs:
for epoch in range(num_epochs):
    
    # Decay the learning rate every 20 epochs:
    if (epoch+1) % 20 == 0:
        decay+=1
        optimizer.param_groups[0]['lr'] = learning_rate * (0.5**decay)
        print("The new learning rate is {}".format(optimizer.param_groups[0]['lr']))
    
    # Iterating over batches of data:
    for i, (images, labels) in enumerate(train_loader):
        # Moving images and labels to the device (GPU or CPU):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass: compute the model output:
        outputs = model(images)
        
        # Compute the loss:
        loss = criterion(outputs, labels)
        
        # Zero the gradients, perform a backward pass, and update the weights:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print loss every 100 steps:
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/25], Step [100/500] Loss: 1.4264
Epoch [1/25], Step [200/500] Loss: 1.4050
Epoch [1/25], Step [300/500] Loss: 1.2772
Epoch [1/25], Step [400/500] Loss: 1.0944
Epoch [1/25], Step [500/500] Loss: 1.1442
Epoch [2/25], Step [100/500] Loss: 1.1027
Epoch [2/25], Step [200/500] Loss: 1.2171
Epoch [2/25], Step [300/500] Loss: 0.9066
Epoch [2/25], Step [400/500] Loss: 1.0499
Epoch [2/25], Step [500/500] Loss: 0.9299
Epoch [3/25], Step [100/500] Loss: 0.9418
Epoch [3/25], Step [200/500] Loss: 1.0341
Epoch [3/25], Step [300/500] Loss: 0.8758
Epoch [3/25], Step [400/500] Loss: 0.7915
Epoch [3/25], Step [500/500] Loss: 1.0323
Epoch [4/25], Step [100/500] Loss: 0.7945
Epoch [4/25], Step [200/500] Loss: 0.8204
Epoch [4/25], Step [300/500] Loss: 0.8019
Epoch [4/25], Step [400/500] Loss: 0.7735
Epoch [4/25], Step [500/500] Loss: 0.9215
Epoch [5/25], Step [100/500] Loss: 0.7730
Epoch [5/25], Step [200/500] Loss: 0.7604
Epoch [5/25], Step [300/500] Loss: 0.7221
Epoch [5/25], Step [400/500] Loss:

In [7]:
# Setting the model to evaluation mode:
model.eval()

# Disabling gradient calculation for efficiency:
with torch.no_grad():
    correct = 0   # Counter for correct predictions
    total = 0     # Counter for total predictions
    
    # Iterating over the test dataset:
    for images, labels in test_loader:
        # Moving images and labels to the configured device (GPU or CPU):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass to get outputs:
        outputs = model(images)
        
        # Getting the predicted class with the highest score:
        _, predicted = torch.max(outputs.data, 1)
        
        # Updating the total and correct counts:
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # Calculating and printing the accuracy:
    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 85.16 %
